# Initialize workspace

In [15]:
from azureml.core import Workspace
ws = Workspace.from_config(path=".file-path/ws_config.json")

# Download and register model

In [16]:
import os
import urllib.request
from azureml.core.model import Model

# Download model
onnx_model_url = "https://www.cntk.ai/OnnxModels/mnist/opset_7/mnist.tar.gz"
urllib.request.urlretrieve(onnx_model_url, filename="mnist.tar.gz")
os.system('tar xvzf mnist.tar.gz')
# Register model
model = Model.register(workspace = ws,
                        model_path ="mnist/model.onnx",
                        model_name = "onnx_mnist",
                        tags = {"onnx": "demo"},
                        description = "MNIST image classification CNN from ONNX Model Zoo",)

#model = Model(ws,'onnx_mnist')

Registering model onnx_mnist


# Create a driver file
To run a model in Azure ML, you need to define a driver file.

In [31]:
with open("onnx/onnx_score.py", "r") as driver_file:
    print(driver_file.read())

import json
import numpy as np
import onnxruntime
import sys
import os
from azureml.core.model import Model
import time


def init():
    global session, input_name, output_name
    model = os.path.join(os.getenv('AZUREML_MODEL_DIR'),'model.onnx')
    session = onnxruntime.InferenceSession(model, None)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name 
    

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    return np.array(json.loads(input_data_json)['data']).astype('float32')

def postprocess(result):
    # We use argmax to pick the highest confidence label
    return int(np.argmax(np.array(result).squeeze(), axis=0))
    
def run(input_data):

    try:
        # load in our data, convert to readable format
        data = preprocess(input_data)
        
        # start timer
        start = time.time()
        
        r = session.run([output_name], {input_name: data})
        
        #end timer
        

# Create inference environment
Inference environments can be specified from pip / conda / a dockerfile.

This example uses conda:

In [25]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
import os

myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-defaults"])

with open(os.path.join("onnx","onnx_env.yml"),"w") as f:
    f.write(myenv.serialize_to_string())
    
deploy_env = Environment.from_conda_specification("onnx_mnist","onnx/onnx_env.yml")
print(deploy_env)

Environment(Name: onnx_mnist,
Version: None)


# Define InferenceConfig

In [26]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(environment=deploy_env, 
                                   entry_script="onnx/onnx_score.py")

# Deploy locally

In [ ]:
from azureml.core.webservice import LocalWebservice
from azureml.core import Model

local_deploy_config = LocalWebservice.deploy_configuration(port=6789)
local_service = Model.deploy(ws, "testlocal", [model], inference_config, local_deploy_config)
local_service.wait_for_deployment()
print('Local service port: {}'.format(local_service.port))


# Deploy remotely

In [17]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'ONNX for mnist model')
aci_service_name = 'onnx-demo-mnist-aci'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service onnx-demo-mnist-aci
Running..............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


# Debugging your service
Deployed services can be debugged by running the below command.

In [22]:
aci_service.get_logs()

"/bin/bash: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n2019-10-01T16:26:28,019934003+00:00 - gunicorn/run \n2019-10-01T16:26:28,017860485+00:00 - iot-server/run \n2019-10-01T16:26:28,021331715+00:00 - rsyslog/run \nbash: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libtinfo.so.5: no version information available (required by bash)\n2019-10-01T16:26:28,026211858+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_92b60d2fdd9f52c2d90c3626147e6f6b/lib/libcrypt

# Test your service

In [ ]:
# ONNX - Predict
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice

import json
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt  
from IPython.display import Image

%matplotlib inline

ws = Workspace.from_config()
aci_service = AciWebservice(ws,'onnx-demo-mnist-2')

def rgb2gray(rgb):
    """Convert the input image into grayscale"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def resize_img(img_to_resize):
    """Resize image to MNIST model input dimensions"""
    r_img = cv2.resize(img_to_resize, dsize=(28, 28), interpolation=cv2.INTER_AREA)
    r_img.resize((1, 1, 28, 28))
    return r_img

def preprocess(img_to_preprocess):
    """Resize input images and convert them to grayscale."""
    if img_to_preprocess.shape == (28, 28):
        img_to_preprocess.resize((1, 1, 28, 28))
        return img_to_preprocess
    
    grayscale = rgb2gray(img_to_preprocess)
    processed_img = resize_img(grayscale)
    return processed_img

your_test_image = "mnist/3.png"
img = cv2.imread(your_test_image)
plt.subplot(1,3,1)
plt.imshow(img, cmap = plt.cm.Greys)
print("Old Dimensions: ", img.shape)
img = preprocess(img)
print("New Dimensions: ", img.shape)

try:
    input_data = json.dumps({'data': img.tolist()})
    r = aci_service.run(input_data)
    time_ms = np.round(r['time_in_sec'] * 1000, 2)
    print(r)
except KeyError as e:
    print(str(e))

plt.figure(figsize = (16, 6))
plt.subplot(1, 15,1)
plt.axhline('')
plt.axvline('')
plt.text(x = -100, y = -20, s = "Model prediction: ", fontsize = 14)
plt.text(x = -100, y = -10, s = "Inference time: ", fontsize = 14)
plt.text(x = 0, y = -20, s = str(r["result"]), fontsize = 14)
plt.text(x = 0, y = -10, s = str(time_ms) + " ms", fontsize = 14)
plt.text(x = -100, y = 14, s = "Input image: ", fontsize = 14)
plt.imshow(img.reshape(28, 28), cmap = plt.cm.gray)

# Deploy models in other frameworks

In [ ]:
# SKLEARN - Register & Deploy
from azureml.core import Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

sklearn_model = Model.register(ws,"sklearn/model.pkl","sklearn-model")
sklearn_env = Environment.from_conda_specification("sklearn_mnist","sklearn/env.yml")
inference_config = InferenceConfig(environment=sklearn_env, 
                                   entry_script="sklearn/score.py")

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1)

sklearn_service = Model.deploy(ws, "sklearn-svc", [sklearn_model], inference_config, aciconfig)
sklearn_service.get_logs()

In [53]:
# SKLEARN - Predict
import json
from azureml.core.webservice import AciWebservice

sklearn_service = AciWebservice(ws,"sklearn-svc")

test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})

test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = sklearn_service.run(input_data=test_sample_encoded)

print(prediction)

[5215.1981315798685, 3726.995485938578]
